AlgoTrading回测工具使用
==============
***

1. 策略描述
------------------
***

定义的策略需要继承自``strategy``基类，并且实现两个成员方法：

* ``__init__``，*策略初始化*
* ``handle_data``，*每次新的市场数据到来时运行*

In [ ]:
%matplotlib inline
import datetime as dt
from AlgoTrading.Strategy import Strategy
from AlgoTrading.Backtest import strategyRunner
from AlgoTrading.Backtest import DataSource
from PyFin.API import MA

策略信号：

* ``MA(10, 'close')``：过去10条bar的收盘价的均线；
* ``MA(30, 'close')``：过去30条bar的收盘价均线；
* 当``MA(10, 'close')``高于``MA(30, 'close')``时，买入；
* 当``MA(10, 'close')``低于``MA(30, 'close')``时，卖出。

In [ ]:
class MovingAverageCrossStrategy(Strategy):
    def __init__(self):
        self.signal = MA(10, 'close') - MA(30, 'close')

    def handle_data(self):
        for s in self.universe:
            if self.signal[s] > 0.0 and self.secPos[s] == 0:
                self.order(s, 1, quantity=1000)
            elif self.signal[s] < 0.0 and self.secPos[s] != 0:
                self.order(s, -1, quantity=1000)

2. 策略运行设置
------------------
***

需要设置策略运行的基本参数:

* ``universe``: *证券池*
* ``initialCapital``：*初始资金*
* ``startDate``：*回测开始日期*
* ``endDate``：*回测结束日期*
* ``dataSource``：*数据源*
* ``freq``：回测时间频率（分钟），仅当使用``DataSource.DXDataCenter``时有效。

In [ ]:
def run_example():
    universe = ['000001.XSHE', '000002.XSHE', '000004.XSHE', '000005.XSHE', '000006.XSHE', '000007.XSHE', '000008.XSHE']
    initialCapital = 100000.0
    startDate = dt.datetime(2011, 10, 1)
    endDate = dt.datetime(2015, 10, 1)

    strategyRunner(userStrategy=MovingAverageCrossStrategy,
                   initialCapital=initialCapital,
                   symbolList=universe,
                   startDate=startDate,
                   endDate=endDate,
                   dataSource=DataSource.DXDataCenter,
                   freq=1,
                   saveFile=True,
                   plot=True)

3. 策略运行并输出
----------------
***

策略运行之后会在当前页面下做两件事：

* *输出策略的整体表现至图表，并在当前页面下显示*
* *输出策略表现的具体数据至excel表格，默认为当前目录下的* ```performance.xlsx```

In [ ]:
run_example()

4. 获取表现的表格数据
----------------------
***

In [ ]:
import pandas as pd
wb = pd.read_excel('performance.xlsx', sheetname=None)

4.1 策略整体表现
-------------------

In [ ]:
wb['perf_metrics']

4.2 策略表现历史时间序列
-------------------------

In [ ]:
wb['perf_series'].tail()

4.3 策略表现（分证券）
------------------

In [ ]:
wb['equity_curve'].tail()

4.4 策略指令簿
------------------

In [ ]:
wb['order_book'].tail()

4.5 策略成交簿
----------------

In [ ]:
wb['filled_book'].tail()